## Parcial 1 Métodos numéricos 2
### Sección 10
Javier Alejandro Ovalle Chiquín, 22103  
Ricardo Josué Morales Contreras, 22289 

#### Probelma 1, responder si la afirmación es falsa 0 verdadera

1. M admite factorización LU  
2. M admite factorización PA = LU  
3. M admite factorización SVD
4. M admite factorización LL^t  
5. M admite factorización QR

In [ ]:
# Ejercicio 1: factorizaciones para M
import numpy as np

np.set_printoptions(precision=6, suppress=True)

# Matriz del enunciado
M = np.array([[1., 1., 0.],
              [0., 1., 1.],
              [0., 0., 1.]])

print("M =\n", M)

# ---------- Utilidades ----------
def leading_principal_minors_nonzero(A):
    """Chequea si todos los menores principales líderes son no nulos."""
    for k in range(1, A.shape[0]+1):
        if np.linalg.det(A[:k, :k]) == 0:
            return False
    return True

def lu_nopivot(A):
    """Doolittle (sin pivoteo). Retorna L, U si es posible, si no lanza ValueError."""
    A = A.copy().astype(float)
    n = A.shape[0]
    L = np.zeros_like(A)
    U = np.zeros_like(A)
    for i in range(n):
        L[i, i] = 1.0
    for k in range(n):
        # Pivote cero -> no se puede sin pivoteo
        if abs(A[k, k]) < 1e-15:
            raise ValueError("Pivote cero: LU sin pivoteo no es posible")
        # U fila k
        for j in range(k, n):
            U[k, j] = A[k, j] - L[k, :k] @ U[:k, j]
        # L columna k
        for i in range(k+1, n):
            L[i, k] = (A[i, k] - L[i, :k] @ U[:k, k]) / U[k, k]
    return L, U

def lu_with_partial_pivot(A):
    """LU con pivoteo parcial: PA = LU. Retorna P, L, U."""
    A = A.copy().astype(float)
    n = A.shape[0]
    P = np.eye(n)
    L = np.zeros_like(A)
    U = A.copy()
    for i in range(n):
        # Pivoteo parcial
        pivot = np.argmax(np.abs(U[i:, i])) + i
        if abs(U[pivot, i]) < 1e-15:
            raise ValueError("Matriz singular: pivote cero con pivoteo parcial.")
        # Intercambiar filas en U y P, y en L (columnas previas)
        if pivot != i:
            U[[i, pivot], i:] = U[[pivot, i], i:]
            P[[i, pivot], :] = P[[pivot, i], :]
            if i > 0:
                L[[i, pivot], :i] = L[[pivot, i], :i]
        # Eliminación
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= L[j, i] * U[i, i:]
    np.fill_diagonal(L, 1.0)
    return P, L, U

def is_spd(A):
    """Chequea si es simétrica definida positiva."""
    if not np.allclose(A, A.T, atol=1e-12):
        return False
    # Autovalores todos positivos
    w = np.linalg.eigvalsh(A)
    return np.all(w > 1e-12)

# ---------- (a) LU (sin pivoteo) ----------
adm_lu = False
lu_reason = ""
try:
    # Criterio práctico: pivotes diagonales no nulos en triangular => existe
    L, U = lu_nopivot(M)
    ok_recon = np.allclose(L @ U, M, atol=1e-12)
    adm_lu = ok_recon
    lu_reason = "Triangular superior con diagonal no nula → pivotes 1,1,1 → LU sin pivoteo existe."
except ValueError as e:
    lu_reason = f"No: {e}"

print("\n(a) ¿M admite LU (sin pivoteo)?", adm_lu)
print("Justificación:", lu_reason)

# ---------- (b) PA = LU (con pivoteo parcial) ----------
adm_pa_lu = False
pa_lu_reason = ""
try:
    P, Lp, Up = lu_with_partial_pivot(M)
    ok_pa = np.allclose(P @ M, Lp @ Up, atol=1e-10)
    adm_pa_lu = ok_pa
    pa_lu_reason = "LU con pivoteo parcial siempre existe para matrices no singulares; verificación numérica OK."
except ValueError as e:
    pa_lu_reason = f"No: {e}"

print("\n(b) ¿M admite PA = LU (pivoteo parcial)?", adm_pa_lu)
print("Justificación:", pa_lu_reason)

# ---------- (c) SVD ----------
# Siempre existe para cualquier matriz real.
U, s, Vt = np.linalg.svd(M, full_matrices=False)
svd_ok = np.allclose(U @ np.diag(s) @ Vt, M, atol=1e-10)
print("\n(c) ¿M admite SVD?", svd_ok)
print("Justificación: SVD existe para cualquier matriz real; reconstrucción compacta correcta. "
      f"singulares = {s}")

# ---------- (d) LL^T (Cholesky) ----------
adm_chol = False
chol_reason = ""
if is_spd(M):
    adm_chol = True
    chol_reason = "M es simétrica definida positiva."
else:
    chol_reason = "No: no es simétrica definida positiva (M ≠ M^T)."
print("\n(d) ¿M admite L L^T (Cholesky)?", adm_chol)
print("Justificación:", chol_reason)

# ---------- (e) QR ----------
# Siempre existe (Gram-Schmidt / Householder)
Q, R = np.linalg.qr(M)
qr_ok = np.allclose(Q @ R, M, atol=1e-10)
print("\n(e) ¿M admite QR?", qr_ok)
print("Justificación: descomposición QR siempre existe (Householder/Gram-Schmidt); verificación numérica OK.")


M =
 [[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]]

(a) ¿M admite LU (sin pivoteo)? True
Justificación: Triangular superior con diagonal no nula → pivotes 1,1,1 → LU sin pivoteo existe.

(b) ¿M admite PA = LU (pivoteo parcial)? True
Justificación: LU con pivoteo parcial siempre existe para matrices no singulares; verificación numérica OK.

(c) ¿M admite SVD? True
Justificación: SVD existe para cualquier matriz real; reconstrucción compacta correcta. singulares = [1.801938 1.24698  0.445042]

(d) ¿M admite L L^T (Cholesky)? False
Justificación: No: no es simétrica definida positiva (M ≠ M^T).

(e) ¿M admite QR? True
Justificación: descomposición QR siempre existe (Householder/Gram-Schmidt); verificación numérica OK.


## Ejercicio 2 – SVD reducida y Vᵀ

En clase vimos que la descomposición en valores singulares (SVD) reducida de una matriz 
\(A \in \mathbb{R}^{m \times n}\) es de la forma:


A = U \Sigma Vᵀ


donde:
- \(U \in \mathbb{R}^{m \times k}\) con \(k = \min(m,n)\),
- \(\Sigma \in \mathbb{R}^{k \times k}\) es diagonal con los valores singulares \(\sigma_i\),
- \(V^\top \in \mathbb{R}^{k \times n}\).

En este ejercicio trabajamos con la matriz:

\[
A =
\begin{bmatrix}
1 & 5 & 3 & 2 & 4 \\
2 & 2 & 0 & 4 & 1 \\
2 & 1 & 8 & 4 & 2
\end{bmatrix}.
\]

Al aplicar la función de Python:

```python
np.linalg.svd(A, full_matrices=False)




---
### (a) ¿Por qué no se obtienen matrices $S$ y $V^\top$ de tamaño $5 \times 5$?

La matriz $A$ es de tamaño $3 \times 5$.  
Cuando usamos `full_matrices=False`, NumPy devuelve la **SVD reducida**, donde:

- $U \in \mathbb{R}^{3 \times 3}$,
- $\Sigma \in \mathbb{R}^{3 \times 3}$,
- $V^\top \in \mathbb{R}^{3 \times 5}$.

Esto ocurre porque la SVD reducida utiliza:

$$
k = \min(m,n) = \min(3,5) = 3,
$$

y solo genera las columnas/filas necesarias para reconstruir $A$.

Conclusión: no aparecen matrices $5 \times 5$ porque la versión reducida **omite la parte sobrante** de $V^\top$ (y las dimensiones extra de $\Sigma$).


### (b.1) Cálculo de $V^\top$ a mano (con todos los pasos)

Queremos obtener $V^\top$ de la descomposición SVD reducida de

$$
A=
\begin{bmatrix}
1 & 5 & 3 & 2 & 4 \\
2 & 2 & 0 & 4 & 1 \\
2 & 1 & 8 & 4 & 2
\end{bmatrix}\in\mathbb{R}^{3\times 5}.
$$

Recordemos que:

$$
A^\top A = V\,\Sigma^2\,V^\top.
$$

---

#### **Paso 1. Construir $A^\top A$**

Multiplicamos:

$$
A^\top A =
\begin{bmatrix}
9 & 9 & 19 & 14 & 12 \\
9 & 30 & 31 & 24 & 28 \\
19 & 31 & 73 & 42 & 42 \\
14 & 24 & 42 & 36 & 20 \\
12 & 28 & 42 & 20 & 21
\end{bmatrix}.
$$

---

#### **Paso 2. Calcular los autovalores de $A^\top A$**

Resolviendo $\det(A^\top A - \lambda I)=0$, se obtienen los autovalores (ordenados descendentemente):

$$
\lambda_1 \approx 132.41523, \quad
\lambda_2 = 25, \quad
\lambda_3 \approx 11.58477, \quad
\lambda_4 = 0, \quad
\lambda_5 = 0.
$$

---

#### **Paso 3. Relación entre autovalores y valores singulares**

Recordemos:  
$$
\sigma_i = \sqrt{\lambda_i}.
$$

Por lo tanto:

$$
\sigma_1 \approx 11.507182, \quad
\sigma_2 = 5, \quad
\sigma_3 \approx 3.403641.
$$

---

#### **Paso 4. Calcular los autovectores de $A^\top A$**

Para cada $\lambda_i > 0$, resolvemos

$$
(A^\top A - \lambda_i I)v_i = 0,
$$

y normalizamos $v_i$ para que tenga norma 1.  
Esto nos da los tres **vectores singulares derechos** principales:

$$
\begin{aligned}
v_1^\top &\approx [-0.234763,\ -0.362728,\ -0.683161,\ -0.469526,\ -0.355177], \\
v_2^\top &\approx [\ 0.052686,\ \ 0.684922,\ -0.632236,\ \ 0.105373,\ \ 0.342461], \\
v_3^\top &\approx [-0.375416,\ \ 0.270858,\ \ 0.322852,\ -0.750832,\ \ 0.343099].
\end{aligned}
$$

---

#### **Paso 5. Completar la base con autovectores del núcleo**

Como $A$ tiene rango 3, hay dos autovalores iguales a 0.  
Sus autovectores corresponden al **núcleo de $A$**, y completan la base ortonormal:

$$
\begin{aligned}
v_4^\top &\approx [-0.870767,\ \ 0.153327,\ \ 0.045998,\ \ 0.412385,\ -0.214658], \\
v_5^\top &\approx [-0.207213,\ -0.549942,\ -0.164983,\ \ 0.186098,\ \ 0.769919].
\end{aligned}
$$

---

#### **Paso 6. Formar $V$ y transponer**

La matriz $V$ se construye con estas columnas:

$$
V = [v_1\ v_2\ v_3\ v_4\ v_5],
$$

y por lo tanto

$$
V^\top \approx
\begin{bmatrix}
-0.234763 & -0.362728 & -0.683161 & -0.469526 & -0.355177 \\
\ \ 0.052686 & \ \ 0.684922 & -0.632236 & \ \ 0.105373 & \ \ 0.342461 \\
-0.375416 & \ \ 0.270858 & \ \ 0.322852 & -0.750832 & \ \ 0.343099 \\
-0.870767 & \ \ 0.153327 & \ \ 0.045998 & \ \ 0.412385 & -0.214658 \\
-0.207213 & -0.549942 & -0.164983 & \ \ 0.186098 & \ \ 0.769919
\end{bmatrix}.
$$

---

#### **Paso 7. Verificación**

1. Los vectores de $V$ son ortonormales ($V^\top V=I$).  
2. Con $\Sigma=\operatorname{diag}(\sigma_1,\sigma_2,\sigma_3,0,0)$ se cumple:

$$
A^\top A = V \Sigma^2 V^\top,
$$

y por la definición de SVD:

$$
A = U \Sigma V^\top.
$$


In [12]:
# Matriz A (3x5)
A = np.array([
    [1., 5., 3., 2., 4.],
    [2., 2., 0., 4., 1.],
    [2., 1., 8., 4., 2.]
])

# ---- Opción 1: SVD completa
U_full, s_full, Vt_full = np.linalg.svd(A, full_matrices=True)
print("Valores singulares (σ):", s_full)
print("V^T completo (5x5):\n", Vt_full)

# ---- Opción 2: Eigen de A^T A
ATA = A.T @ A
eigvals, eigvecs = np.linalg.eigh(ATA)
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]
Vt_from_eigh = eigvecs.T
print("\nAutovalores de A^T A (desc):", eigvals)
print("V^T (5x5) vía eigen:\n", Vt_from_eigh)

# ---- Opción 3: Relación v_i = (A^T u_i)/σ_i
U_red, s_red, Vt_red = np.linalg.svd(A, full_matrices=False)
V_from_relation = (A.T @ U_red) / s_red
Vt_from_relation = V_from_relation.T
print("\nV^T reducido (3x5) vía relación (A^T U)/σ:\n", Vt_from_relation)


Valores singulares (σ): [11.507181664855  5.              3.403640717236]
V^T completo (5x5):
 [[-0.234762846438 -0.362728440875 -0.683161270311 -0.469525692875
  -0.355177120026]
 [ 0.052686336977  0.684922380706 -0.632236043729  0.105372673955
   0.342461190353]
 [-0.375415973235  0.270858209012  0.322851656414 -0.75083194647
   0.343098927076]
 [-0.870767491377  0.153326844118  0.045998053235  0.412384719071
  -0.214657581765]
 [-0.20721336547  -0.549941924272 -0.164982577282  0.186097971376
   0.769918693981]]

Autovalores de A^T A (desc): [132.415229867973  25.              11.584770132027   0.
  -0.            ]
V^T (5x5) vía eigen:
 [[-0.234762846438 -0.362728440875 -0.683161270311 -0.469525692875
  -0.355177120026]
 [ 0.052686336977  0.684922380706 -0.632236043729  0.105372673955
   0.342461190353]
 [-0.375415973235  0.270858209012  0.322851656414 -0.75083194647
   0.343098927076]
 [-0.124556058096  0.567987663359  0.170396299008 -0.022920120456
  -0.795182728702]
 [-0.88637418

## Problema 3

Hallar la matriz inversa de

$
T =
\begin{pmatrix}
-1 & -1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & -1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & -1 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & -1 & 1
\end{pmatrix}
$

de forma eficiente.
¿Cuál es la complejidad computacional del algoritmo utilizado?

In [10]:
# Problema 3 - Inversa de T por LU con pivoteo parcial

np.set_printoptions(precision=6, suppress=True)

# Definir la matriz T (6x6)
T = np.array([
    [-1., -1.,  0.,  0.,  0.,  0.],
    [ 1.,  0.,  0., -1.,  0.,  0.],
    [ 0.,  1.,  0.,  0.,  0., -1.],
    [ 0.,  0.,  1.,  0.,  0.,  0.],
    [ 0.,  0., -1.,  0.,  1.,  0.],
    [ 0.,  0.,  0.,  0., -1.,  1.],
])

# --- LU con pivoteo parcial ---
def lu_with_partial_pivot(A):
    A = A.copy().astype(float)
    n = A.shape[0]
    P = np.eye(n)
    L = np.zeros_like(A)
    U = A.copy()
    for i in range(n):
        # Buscar pivote
        pivot = np.argmax(np.abs(U[i:, i])) + i
        if np.isclose(U[pivot, i], 0.0):
            raise ValueError("Matriz singular.")
        # Intercambiar filas
        if pivot != i:
            U[[i, pivot], i:] = U[[pivot, i], i:]
            P[[i, pivot], :] = P[[pivot, i], :]
            if i > 0:
                L[[i, pivot], :i] = L[[pivot, i], :i]
        # Eliminación
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= L[j, i] * U[i, i:]
    np.fill_diagonal(L, 1.0)
    return P, L, U

def forward_substitution(L, b):
    y = b.astype(float).copy()
    for i in range(L.shape[0]):
        y[i] -= L[i, :i] @ y[:i]
    return y  # diag(L)=1

def back_substitution(U, y):
    x = y.astype(float).copy()
    n = U.shape[0]
    for i in range(n-1, -1, -1):
        x[i] -= U[i, i+1:] @ x[i+1:]
        x[i] /= U[i, i]
    return x

# Factorizar una vez
P, L, U = lu_with_partial_pivot(T)

# Resolver T X = I (para obtener T^{-1})
I = np.eye(T.shape[0])
cols = []
for k in range(T.shape[0]):
    pb = P @ I[:, k]
    y  = forward_substitution(L, pb)
    x  = back_substitution(U, y)
    cols.append(x)

T_inv = np.column_stack(cols)

print("Matriz T =\n", T)
print("\nMatriz inversa T^{-1} =\n", T_inv)
print("\nVerificación ||T T^{-1} - I||_F =", np.linalg.norm(T @ T_inv - I, ord='fro'))


Matriz T =
 [[-1. -1.  0.  0.  0.  0.]
 [ 1.  0.  0. -1.  0.  0.]
 [ 0.  1.  0.  0.  0. -1.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0. -1.  0.  1.  0.]
 [ 0.  0.  0.  0. -1.  1.]]

Matriz inversa T^{-1} =
 [[-1. -0. -1. -1. -1. -1.]
 [-0. -0.  1.  1.  1.  1.]
 [ 0.  0.  0.  1.  0.  0.]
 [-1. -1. -1. -1. -1. -1.]
 [ 0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  1.  1.  1.]]

Verificación ||T T^{-1} - I||_F = 0.0


### Problema 3 — Inversa de $T$

Lo que hicimos aquí fue sacar la inversa de la matriz $T$ usando una **factorización LU con pivoteo parcial**.  
La idea no es calcular la inversa “a lo bruto” (que sería más costoso), sino factorizar $T$ una sola vez como $LU$ y luego resolver $T X = I$ para ir sacando cada columna de la inversa.  

Al final nos dio una $T^{-1}$ que cumple perfecto:

$$
\| T T^{-1} - I \|_F = 0,
$$

así que sabemos que está bien hecha.

---

### Complejidad

- La parte pesada es la **LU**, que cuesta $O(n^3)$.  
- Después, resolver $n$ sistemas triangulares (uno por cada columna de la identidad) cuesta en total también $O(n^3)$.  
- En resumen: todo el proceso sigue siendo $O(n^3)$.


#%% md
# Problema 4

Suponga que se tiene un conjunto de 12 pares de datos $\{(x_i, y_i)\}_{i=1}^{12}$, donde $x_i \in \mathbb{R}^3$ y $y_i \in \mathbb{R}$.

Se desea construir un modelo de regresión lineal de la forma

$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \epsilon,$$

donde $\beta = (\beta_0, \beta_1, \beta_2, \beta_3)^T$ es el vector de coeficientes del modelo, y $x = (x_1, x_2, x_3)^T \in \mathbb{R}^3$ son las observaciones, y el término $\epsilon \sim \mathcal{N}(0, \sigma)$ es la variable aleatoria del ruido asociado a la incerteza.

Para ello, se construye la matriz de diseño del modelo

$$
\mathbf{X} =
\begin{pmatrix}
1 & 0.001 & 1 & 3 \\
1 & 0.002 & 1.001 & 3 \\
1 & 0.003 & 1.002 & 3 \\
1 & 0.004 & 1 & 3.001 \\
1 & 0.005 & 1.001 & 3.001 \\
1 & 0.006 & 1.002 & 3.001 \\
1 & 0.006 & 1 & 3.002 \\
1 & 0.005 & 1.001 & 3.002 \\
1 & 0.004 & 1.002 & 3.002 \\
1 & 0.003 & 1 & 3.003 \\
1 & 0.002 & 1.001 & 3.003 \\
1 & 0.001 & 1.002 & 3.003
\end{pmatrix},
$$

y el vector de observaciones $y = (y_1, y_2, \ldots, y_{12})^T$.

Se sabe que los coeficientes óptimos (el estimador de máxima verosimilitud del modelo de regresión), $\hat{\beta}$, están dados por

$$(\mathbf{X}^T \mathbf{X})^{-1} \hat{\beta} = \mathbf{X}^T \mathbf{y},$$

y la varianza del coeficiente $\hat{\beta}_j$ es proporcional a

$$\text{Var}(\hat{\beta}j) = \sigma^2 C{jj},$$

donde $C_{jj}$ es la j–ésima entrada en la diagonal de la matriz $C = (\mathbf{X}^T \mathbf{X})^{-1}$. Con base en lo anterior

a) Hallar el número de condición del sistema.

b) ¿El sistema es estable? ¿Por qué?

c) ¿Cuáles son las causas de lo anterior?

d) Asumiendo $\sigma = 1$, ¿Cuáles son las varianzas de cada coeficiente?
    ¿Qué puede inferir sobre los valores de los coeficientes y sobre la calidad de su modelo de regresión?

In [11]:
# Problema 4 - Análisis de condición y varianzas en regresión lineal
np.set_printoptions(precision=12, suppress=True)

# Matriz de diseño X (12x4)
X = np.array([
    [1, 0.001, 1.000, 3.000],
    [1, 0.002, 1.001, 3.000],
    [1, 0.003, 1.002, 3.000],
    [1, 0.004, 1.000, 3.001],
    [1, 0.005, 1.001, 3.001],
    [1, 0.006, 1.002, 3.001],
    [1, 0.006, 1.000, 3.002],
    [1, 0.005, 1.001, 3.002],
    [1, 0.004, 1.002, 3.002],
    [1, 0.003, 1.000, 3.003],
    [1, 0.002, 1.001, 3.003],
    [1, 0.001, 1.002, 3.003],
], dtype=float)

# (a) Número de condición de X y de X^T X
U, s, Vt = np.linalg.svd(X, full_matrices=False)
kappa_X = s.max() / s.min()
XtX = X.T @ X
eigvals = np.linalg.eigvalsh(XtX)
kappa_XtX = eigvals.max() / eigvals.min()

print("Valores singulares de X:", s)
print("Número de condición kappa(X):", kappa_X)
print("Autovalores de X^T X:", eigvals)
print("Número de condición kappa(X^T X):", kappa_XtX)

# (d) Varianzas de los coeficientes (sigma^2=1)
C = np.linalg.inv(XtX)
variances = np.diag(C)
print("\nVarianzas de los coeficientes (sigma^2=1):", variances)


Valores singulares de X: [11.494877076365  0.005916077777  0.002958781936  0.001115742827]
Número de condición kappa(X): 10302.443181954317
Autovalores de X^T X: [  0.000001244882   0.000008754391   0.000034999976 132.132199000751]
Número de condición kappa(X^T X): 106140334.84471254

Varianzas de los coeficientes (sigma^2=1): [725850.7084675523    28571.428571428543 125000.00001028333
  66666.66665621131 ]


### Interpretación de los resultados

- **Valores singulares de X:**  
  Salieron `[11.49, 0.0059, 0.0029, 0.0011]`.  
  Esto muestra una diferencia enorme entre el mayor y el menor ⇒ algunas direcciones de los datos aportan mucha información y otras son casi redundantes.  

- **Número de condición de X:**  
  $\kappa(X) \approx 1.03 \times 10^4$.  
  Es altísimo: cualquier error pequeño en los datos puede amplificarse miles de veces en los coeficientes.  

- **Autovalores de $X^\top X$:**  
  `[1.25e-6, 8.75e-6, 3.5e-4, 132.13]`.  
  Varios son casi cero ⇒ $X^\top X$ es casi singular, lo que confirma problemas de multicolinealidad.  

- **Número de condición de $X^\top X$:**  
  $\kappa(X^\top X) \approx 1.06 \times 10^8$.  
  Recordemos que la condición se **cuadra** cuando trabajamos con ecuaciones normales, por eso es mucho peor que la de $X$.  

- **Varianzas de los coeficientes (σ² = 1):**  
  $[725{,}850.7,\; 28{,}571.4,\; 125{,}000,\; 66{,}666.7]$.  
  Son enormes ⇒ hay muchísima incertidumbre en los coeficientes estimados, los intervalos de confianza serían muy anchos y el modelo es poco confiable.

---

**Conclusión:** la matriz de diseño está mal condicionada, lo que vuelve inestable la regresión.  
En la práctica se recomendaría **escalar/centrar variables**, usar **QR/SVD** en lugar de ecuaciones normales o aplicar **regularización (ridge)** para mejorar la estabilidad.
